In [1]:
import netCDF4 as nc
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cmocean as cmo
import os,sys,fnmatch,time
import numpy as np
import matplotlib.gridspec as gridspec
from matplotlib import animation
from salishsea_tools.nc_tools import scDataset
from matplotlib import colors
from matplotlib import ticker
from matplotlib.colors import LinearSegmentedColormap
from mpl_toolkits.axes_grid1.inset_locator import InsetPosition
import matplotlib as mpl
import numpy.ma as ma
from scipy import interpolate
import matplotlib.gridspec as gridspec

%matplotlib inline

import sys
sys.path.append('/ocean/imachuca/Canyons/analysis_mackenzie_canyon/notebooks/upwelling_depth/')
import upwelling_functions
sys.path.append('/ocean/imachuca/Canyons/analysis_mackenzie_canyon/notebooks/general_circulation/')
import general_functions
import quicklook

import warnings
warnings.filterwarnings("ignore")

import time
from datetime import datetime 

/home/imachuca/anaconda3/envs/gsw/lib/python3.6/site-packages/cmocean/tools.py:76: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if not mpl.cbook.is_string_like(rgbin[0]):


In [2]:
def get_varbs(dirname, fname, meshmaskname, time_s):
    
    filesU = general_functions.get_files(dirname, fname, 'grid_U') 
    filesV = general_functions.get_files(dirname, fname, 'grid_V')
    filesW = general_functions.get_files(dirname, fname, 'grid_W')
    filesT = general_functions.get_files(dirname, fname, 'grid_T')
    print('files')
    
    y,x = slice(1,-1,None), slice(1,-1,None)

    with scDataset(filesU) as dsU, scDataset(filesV) as dsV, scDataset(filesW) as dsW, scDataset(filesT) as dsT:
        vozocrtx0 = dsU.variables['vozocrtx'][time_s,:,y,x]
        vomecrty0 = dsV.variables['vomecrty'][time_s,:,y,x]
        vovecrtz0 = dsW.variables['vovecrtz'][time_s,:,y,x]
        vosaline0 = dsT.variables['vosaline'][time_s, :, y, x]
        sozotaux = dsU.variables['sozotaux'][time_s,0,0]
        deptht = dsT.variables['deptht'][:]

    with nc.Dataset(os.path.join(dirname, meshmaskname), 'r') as dsM:
        umask = dsM.variables['umask'][0,:,y,x]
        vmask = dsM.variables['vmask'][0,:,y,x]
        tmask = dsM.variables['tmask'][0,:,y,x]

    vozocrtx = np.ma.array(vozocrtx0, mask=1 - umask)
    vomecrty = np.ma.array(vomecrty0, mask=1 - vmask)
    vovecrtz = np.ma.array(vovecrtz0, mask=1 - tmask)
    vosaline = np.ma.array(vosaline0, mask=1 - tmask)
    
    return vozocrtx, vomecrty, vovecrtz, vosaline, tmask, umask, deptht, sozotaux

In [3]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx, array[idx]

def get_varb_surf(vosaline_array, salt, varb):
    
    if len(vosaline_array.shape)==1:
        f = interpolate.interp1d(vosaline_array, varb)
        ynew = f(salt)
        varb_array = ynew
        
    elif len(vosaline_array.shape)!=1:
        varb_array = np.full([vosaline_array.shape[-2], vosaline_array.shape[-1]], np.nan)
        for i in range(vosaline_array.shape[-1]):
            for j in range(vosaline_array.shape[-2]):
                vosaline_column = vosaline_array[:, j, i]
                if len(varb.shape)==1:
                    varb_column = varb
                elif len(varb.shape)==3:
                    varb_column = varb[:, j, i]

                try:
                    f = interpolate.interp1d(vosaline_column, varb_column)
                    ynew = f(salt)
                    varb_array[j, i] = ynew
                except ValueError:
                    varb_array[j, i] = np.nan  
                
    return varb_array

In [4]:
salt_UHW_SA, salt_ATW_SA = 33.26258156425155, 34.99102991139694
case = 'base'

vosaline_ref = nc.Dataset('/ocean/imachuca/Canyons/mackenzie_canyon/conditions/NEMO_files/salinity/salinity_for_agrif.nc')['vosaline'][:]
vosaline_ref_profile = vosaline_ref[0, :, 0, 0]

time_s=36

In [ ]:
fig, axes = plt.subplots(2, 2, sharex=True, sharey = True, figsize=(20, 18))

kinds = ['ideal', 'ideal', 'real', 'real']
salts = [salt_UHW_SA, salt_ATW_SA, salt_UHW_SA, salt_ATW_SA]
for ax, kind, salt in zip(axes.flatten(), kinds, salts):
    
    dirname = '/ocean/imachuca/Canyons/results_mackenzie/extended_domain/'+kind+'_'+case+'/'
    
    fname = '1_MCKNZ*'
    meshmaskname = '1_mesh_mask.nc'

    vmin, vmax=-0.0008, 0.0008

    
    cmap = cmo.cm.balance
    cmap.set_bad('silver')

    vozocrtx, vomecrty, vovecrtz, vosaline, tmask, umask, deptht, sozotaux = get_varbs(dirname, fname, 
                                                                                   meshmaskname, time_s)

    depth_initial = get_varb_surf(vosaline_ref_profile, salt, deptht)
    w_daily = get_varb_surf(vosaline, salt, vovecrtz)
    print(time_s+1, np.nanmin(w_daily), np.nanmax(w_daily))
    
    u_daily = get_varb_surf(vosaline, salt, vozocrtx)
    v_daily = get_varb_surf(vosaline, salt, vomecrty)
    x_slice, y_slice, u_nstg, v_nstg, speeds = quicklook.get_speeds(u_daily, v_daily, 1)
    strm = ax.streamplot(x_slice, y_slice, u_nstg, v_nstg, color='k', density=2, linewidth=0.5)
    
    xs = np.arange(w_daily.shape[1])
    ys = np.arange(w_daily.shape[0])
    p = ax.pcolormesh(xs, ys, w_daily, cmap=cmap, vmin=vmin, vmax=vmax)
    ax.set_xlim([0, xs[-1]])
    ax.set_ylim([0, ys[-1]])

files


KeyboardInterrupt: 